In [19]:
# Setup and Imports
%load_ext autoreload
%autoreload 2

import sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from uncertainties import ufloat
from matplotlib.lines import Line2D
from matplotlib.patches import Patch

# Add the src directory to import snowpyt_mechparams
sys.path.append('../src')
from snowpilot_utils import convert_grain_form, parse_sample_pits
from snowpyt_mechparams import density, elastic_modulus, shear_modulus, poissons_ratio


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
# Parse all snowpit files from the data folder
all_pits = parse_sample_pits('data')


Successfully parsed 50278 files
Failed to parse 0 files


In [21]:
# Calculate densities for all layers using kim_geldsetzer method
layer_info = []

for pit in all_pits:
    for layer in pit.snow_profile.layers:
        layer_dict = {
            'pit_id': pit.core_info.pit_id,
            'hand_hardness': layer.hardness,
            'depth_top': layer.depth_top[0] if layer.depth_top else None,  
            'thickness': layer.thickness[0] if layer.thickness else None,
        }

        # Convert grain form and calculate density
        grain_form = convert_grain_form(layer.grain_form_primary, 'kim_geldsetzer') if layer.grain_form_primary else None
        layer_dict['kim_geldsetzer_grain_form'] = grain_form
        layer_dict['main_grain_form'] = layer.grain_form_primary.basic_grain_class_code if layer.grain_form_primary else None
        
        # Calculate density if we have valid inputs
        if layer.hardness and grain_form:
            try:
                density_ufloat = density.calculate_density( 
                    method='kim_geldsetzer',
                    hand_hardness=layer.hardness,
                    grain_form=grain_form
                )
                layer_dict['density_kim_geldsetzer'] = density_ufloat.nominal_value
                layer_dict['density_kim_geldsetzer_uncertainty'] = density_ufloat.std_dev
            except Exception:
                layer_dict['density_kim_geldsetzer'] = None
                layer_dict['density_kim_geldsetzer_uncertainty'] = None
        else:
            layer_dict['density_kim_geldsetzer'] = None
            layer_dict['density_kim_geldsetzer_uncertainty'] = None

        layer_info.append(layer_dict)

# Create dataframe
layer_df = pd.DataFrame(layer_info)


In [22]:
# Calculate elastic modulus and shear modulus using wautier method

# Function to calculate elastic modulus and shear modulus for a single row
def calculate_modulii_row(row):
    result = {'e_mod_wautier': None, 's_mod_wautier': None}
    
    # Skip if no grain form or density data
    if pd.isna(row['kim_geldsetzer_grain_form']) or pd.isna(row['density_kim_geldsetzer']):
        return pd.Series(result)
    
    try:
        # Create density ufloat once
        density_ufloat = ufloat(row['density_kim_geldsetzer'], row['density_kim_geldsetzer_uncertainty'])
        grain_form = row['kim_geldsetzer_grain_form']
        
        # Calculate elastic modulus using wautier method
        try:
            e_mod = elastic_modulus.calculate_elastic_modulus(
                method='wautier',
                density=density_ufloat,
                grain_form=grain_form
            )
            result['e_mod_wautier'] = e_mod.nominal_value
        except Exception:
            pass
        
        # Calculate shear modulus using wautier method
        try:
            s_mod = shear_modulus.calculate_shear_modulus(
                method='wautier',
                density=density_ufloat,
                grain_form=grain_form
            )
            result['s_mod_wautier'] = s_mod.nominal_value
        except Exception:
            pass
    except Exception:
        pass
    
    return pd.Series(result)

# Apply the function to all rows at once
layer_df[['e_mod_wautier', 's_mod_wautier']] = layer_df.apply(calculate_modulii_row, axis=1)


/Users/marykateconnelly/Desktop/F25/SnowPyt-MechParams/venv/lib/python3.13/site-packages/uncertainties/core.py:1024: UserWarning: Using UFloat objects with std_dev==0 may give unexpected results.
  warn("Using UFloat objects with std_dev==0 may give unexpected results.")


In [23]:
# Function to calculate poissons ratio for a single row using multiple methods
def calculate_poissons_ratio_row(row):
    """Calculate poissons ratio for a single row using multiple methods."""
    result = {
        'nu_kochle': np.nan,
        'nu_kochle_uncertainty': np.nan,
        'nu_srivastava': np.nan,
        'nu_srivastava_uncertainty': np.nan,
        'nu_from_elastic_and_shear_modulus': np.nan,
        'nu_from_elastic_and_shear_modulus_uncertainty': np.nan
    }
    
    # Skip if no grain form or density data
    if pd.isna(row['main_grain_form']) or pd.isna(row['density_kim_geldsetzer']):
        return pd.Series(result)
    
    try:
        # Create density ufloat once
        density_ufloat = ufloat(row['density_kim_geldsetzer'], row['density_kim_geldsetzer_uncertainty'])
        grain_form = row['main_grain_form']
        
        # Calculate Poisson's ratio using Kochle method
        try:
            nu_kochle = poissons_ratio.calculate_poissons_ratio(
                method='kochle',
                grain_form=grain_form
            )
            result['nu_kochle'] = nu_kochle.nominal_value
            result['nu_kochle_uncertainty'] = nu_kochle.std_dev
        except Exception:
            pass
        
        # Calculate Poisson's ratio using Srivastava method
        try:
            nu_srivastava = poissons_ratio.calculate_poissons_ratio(
                method='srivastava',
                density=density_ufloat,
                grain_form=grain_form
            )
            result['nu_srivastava'] = nu_srivastava.nominal_value
            result['nu_srivastava_uncertainty'] = nu_srivastava.std_dev
        except Exception:
            pass
        
        # Calculate Poisson's ratio from modulii (using wautier E and G)
        # Only calculate if both e_mod_wautier and s_mod_wautier are available
        if not pd.isna(row['e_mod_wautier']) and not pd.isna(row['s_mod_wautier']):
            try:
                # Create ufloats for the moduli (assuming no uncertainty stored)
                e_mod_ufloat = ufloat(row['e_mod_wautier'], 0)
                s_mod_ufloat = ufloat(row['s_mod_wautier'], 0)
                
                nu_from_elastic_and_shear_modulus = poissons_ratio.calculate_poissons_ratio(
                    method='from_elastic_and_shear_modulus',
                    elastic_modulus=e_mod_ufloat,
                    shear_modulus=s_mod_ufloat
                )
                result['nu_from_elastic_and_shear_modulus'] = nu_from_elastic_and_shear_modulus.nominal_value
                result['nu_from_elastic_and_shear_modulus_uncertainty'] = nu_from_elastic_and_shear_modulus.std_dev
            except Exception:
                pass
        
    except Exception:
        pass
    
    return pd.Series(result)

# Apply the function to all rows at once
layer_df[['nu_kochle', 'nu_kochle_uncertainty', 'nu_srivastava', 'nu_srivastava_uncertainty', 'nu_from_elastic_and_shear_modulus', 'nu_from_elastic_and_shear_modulus_uncertainty']] = layer_df.apply(calculate_poissons_ratio_row, axis=1)


/Users/marykateconnelly/Desktop/F25/SnowPyt-MechParams/venv/lib/python3.13/site-packages/uncertainties/core.py:1024: UserWarning: Using UFloat objects with std_dev==0 may give unexpected results.
  warn("Using UFloat objects with std_dev==0 may give unexpected results.")


In [24]:
layer_df.to_csv('poissons_ratio_comparison.csv', index=False)


In [30]:
# Create df and filter out NaN values for each method
df_kochle = layer_df[~layer_df['nu_kochle'].isna()]
df_srivastava = layer_df[~layer_df['nu_srivastava'].isna()]
df_from_elastic_and_shear_modulus = layer_df[~layer_df['nu_from_elastic_and_shear_modulus'].isna()]



In [31]:
# Print summary statistics
print("\n" + "="*80)
print("POISSON'S RATIO COMPARISON SUMMARY STATISTICS")
print("="*80)

print("\nTotal number of layers:")
print(f"  Count: {len(layer_df)}")

print("\nKÖCHLE METHOD:")
print(f"Successfully calculated: {len(df_kochle)}")
print(f"Percent of layers with successful calculation: {len(df_kochle) / len(layer_df) * 100:.2f}%")
print(f"Average uncertainty: {df_kochle['nu_kochle_uncertainty'].mean():.4f}")
print(f"Average relative uncertainty: {df_kochle['nu_kochle_uncertainty'].mean() / df_kochle['nu_kochle'].mean() * 100:.2f}%")

print("\nSRIVASTAVA METHOD:")
print(f"Successfully calculated: {len(df_srivastava)}")
print(f"Percent of layers with successful calculation: {len(df_srivastava) / len(layer_df) * 100:.2f}%")
print(f"Average uncertainty: {df_srivastava['nu_srivastava_uncertainty'].mean():.4f}")
print(f"Average relative uncertainty: {df_srivastava['nu_srivastava_uncertainty'].mean() / df_srivastava['nu_srivastava'].mean() * 100:.2f}%")

print("\nFROM WAUTIER ELASTIC AND SHEAR MODULUS METHOD:")
print(f"Successfully calculated: {len(df_from_elastic_and_shear_modulus)}")
print(f"Percent of layers with successful calculation: {len(df_from_elastic_and_shear_modulus) / len(layer_df) * 100:.2f}%")
print(f"Average uncertainty: {df_from_elastic_and_shear_modulus['nu_from_elastic_and_shear_modulus_uncertainty'].mean():.4f}")
print(f"Average relative uncertainty: {df_from_elastic_and_shear_modulus['nu_from_elastic_and_shear_modulus_uncertainty'].mean() / df_from_elastic_and_shear_modulus['nu_from_elastic_and_shear_modulus'].mean() * 100:.2f}%")




POISSON'S RATIO COMPARISON SUMMARY STATISTICS

Total number of layers:
  Count: 371429

KÖCHLE METHOD:
Successfully calculated: 142163
Percent of layers with successful calculation: 38.27%
Average uncertainty: 0.0361
Average relative uncertainty: 25.22%

SRIVASTAVA METHOD:
Successfully calculated: 103557
Percent of layers with successful calculation: 27.88%
Average uncertainty: 0.0157
Average relative uncertainty: 8.84%

FROM WAUTIER ELASTIC AND SHEAR MODULUS METHOD:
Successfully calculated: 203401
Percent of layers with successful calculation: 54.76%
Average uncertainty: 0.0000
Average relative uncertainty: 0.00%


In [32]:
# Print statistics by grain type
print("\nPOISSON'S RATIO BY GRAIN TYPE:")
print("-" * 80)
for method_name, col_name in [('Köchle', 'nu_kochle'), ('Srivastava', 'nu_srivastava'), ('From Moduli', 'nu_from_elastic_and_shear_modulus')]:
    print(f"\n{method_name.upper()}:")
    df_method = layer_df[~layer_df[col_name].isna()]
    for gt in sorted(df_method['main_grain_form'].unique()):
        data = df_method[df_method['main_grain_form'] == gt][col_name]
        print(f"  {gt}: mean={data.mean():.4f}, std={data.std():.4f}, n={len(data)}")

print("\n" + "="*80)



POISSON'S RATIO BY GRAIN TYPE:
--------------------------------------------------------------------------------

KÖCHLE:
  DH: mean=0.0870, std=0.0000, n=11325
  FC: mean=0.1300, std=0.0000, n=72974
  RG: mean=0.1710, std=0.0000, n=57864

SRIVASTAVA:
  DF: mean=0.1320, std=0.0000, n=1796
  DH: mean=0.1700, std=0.0000, n=11325
  FC: mean=0.1700, std=0.0000, n=45623
  PP: mean=0.1320, std=0.0000, n=811
  RG: mean=0.1910, std=0.0000, n=44002

FROM MODULI:
  DF: mean=0.4991, std=0.0433, n=26392
  DH: mean=0.3771, std=0.0157, n=11325
  FC: mean=0.4197, std=0.0574, n=72974
  MF: mean=0.3345, std=0.0097, n=34846
  RG: mean=0.4082, std=0.0567, n=57864

